In [25]:
%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [26]:
import torch
import torch.optim as optim
from tensorboardX import SummaryWriter
import numpy as np
import os
import argparse
import time
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
#import config   ### just need to import config for config.py - if want to take specific method in config then use from config import "name of method"
from utils.checkpoints import CheckpointIO
from mesh_generate import generate 
#from training import training
from torch.utils import data
from models import OccupancyNetwork
from data import dataset
from tqdm import trange,tqdm
from torch.nn import functional as F
from torch import distributions as dist
from utils.common import (
    compute_iou, make_3d_grid
)
from utils import visualize as vis
from collections import defaultdict
from data import dataset
from training import train
import yaml

In [27]:
# import numpy
# print(numpy.get_include())

In [28]:
path="ocnet.yaml"
with open(path, 'r') as f:
    cfg = yaml.load(f, Loader=yaml.SafeLoader)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Shorthands
out_dir = "out/onet"
backup_every = 100000
#exit_after = args.exit_after  ### can check later if we really need the exit after

model_selection_metric = "iou"
# model_selection_sign = 1

# Output directory
if not os.path.exists(out_dir):
    os.makedirs(out_dir)

In [29]:
print(device)

cuda


In [30]:
def collate_remove_none(batch):
    ''' Collater that puts each data field into a tensor with outer dimension
        batch size.

    Args:
        batch: batch
    '''

    batch = list(filter(lambda x: x is not None, batch))
    return data.dataloader.default_collate(batch)

def worker_init_fn(worker_id):
    ''' Worker init function to ensure true randomness.
    '''
    random_data = os.urandom(4)
    base_seed = int.from_bytes(random_data, byteorder="big")
    np.random.seed(base_seed + worker_id)

In [31]:
logger = SummaryWriter(os.path.join(out_dir, 'logs'))

In [32]:
# Dataset
train_dataset = dataset.get_dataset('train', cfg)
val_dataset = dataset.get_dataset('val', cfg)

train_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=64, num_workers=4, shuffle=True,
    collate_fn=collate_remove_none,
    worker_init_fn=worker_init_fn)

val_loader = torch.utils.data.DataLoader(
    val_dataset, batch_size=12, num_workers=4, shuffle=False,
    collate_fn=collate_remove_none,
    worker_init_fn=worker_init_fn)

In [33]:
# For visualizations
vis_loader = torch.utils.data.DataLoader(
    val_dataset, batch_size=10, shuffle=True,
    collate_fn=collate_remove_none,
    worker_init_fn=worker_init_fn)
data_vis = next(iter(vis_loader))

In [34]:
# Model
model= OccupancyNetwork.OccupancyNetwork(device=device)

In [35]:
# import torch.nn as nn
# from torchvision import models
# from utils.common import normalize_imagenet
# class Resnet18(nn.Module):
#     r''' ResNet-18 encoder network for image input.
#     Args:
#         c_dim (int): output dimension of the latent embedding
#         normalize (bool): whether the input images should be normalized
#         use_linear (bool): whether a final linear layer should be used
#     '''

#     def __init__(self, c_dim=256, normalize=True, use_linear=True):
#         super().__init__()
#         self.normalize = normalize
#         self.use_linear = use_linear
#         self.features = models.resnet18(pretrained=True)
#         self.features.fc = nn.Sequential()
#         self.fc = nn.Linear(512, c_dim)

#     def forward(self, x):
#         x = normalize_imagenet(x)
#         net = self.features(x)
#         out = self.fc(net)
#         return out
# x=Resnet18()
# x

In [36]:
# from torchvision import models
# class ResNetFeatures(nn.Module):
#     def __init__(self, model=models.resnet18(pretrained=True)):
#         super(ResNetFeatures, self).__init__()

#         #self.out_channels = 512
#         self.sequential = nn.Sequential(model.conv1,
#                                   model.bn1,
#                                   model.relu,
#                                   model.maxpool,
#                                   model.layer1,
#                                   model.layer2,
#                                   model.layer3,
#                                   model.layer4)   # to get channels of 512 
#         #print("sequential", self.sequential)
#     def forward(self, x):
#         return self.sequential(x)
    
# def frozen_batch_norm(model):
#     for name, module in model.named_modules():
#         if name == 'bn1' :
#             module.requires_grad_ = False
#         if name == 'layer1' or name == 'layer2' or name == 'layer3' or name == 'layer4':
#             for child_name, child_module in module.named_modules():
#                 if (len(child_name) > 1) and child_name[2:4] == 'bn':
#                     child_module.requires_grad_ = False
#     return model

# y=frozen_batch_norm(ResNetFeatures())   # so out put (bs,512,7,7)
# # y(x)

In [37]:
# class C_Model(nn.Module):
#     def __init__(self):
#         super(CustomModel, self).__init__()
#         self.avgpool = nn.AdaptiveAvgPool2d(output_size=(1, 1))  
#         self.fc = nn.Linear(in_features=512, out_features=256, bias=True)

#     def forward(self, x):
#         x = self.avgpool(x) # (batch size, channels, 1, 1)
#         x = torch.flatten(x, 1) #(batch size, channels)- flatten to force all the tensor here to 1D
#         x = self.fc(x)
#         return x

In [38]:
# self.c_layer=C_Model()

In [39]:

# def forward(self, p, inputs): # p is pointloud (bs,2048,3)
#     feature = self.encoder(inputs)   #(bs,512,7,7)
#     c=CustomModel(feature) # (bs,256)
#     p_r =decoder(feature,p,c)
#     return p_r

In [40]:
# self.feature_channels_change=nn.Conv2d(512,256,1,1)
# self.fc_p1 = nn.Conv1d(3, 256, 1)
# self.bn = CBatchNorm1d(c_dim, hidden_size)
# self.fc_out = nn.Conv1d(hidden_size, 1, 1)
# self.actvn = F.relu
# self.before_last_layer=nn.Sequential(nn.Conv1d(512, 256, 1))
# def fc_p(p_size,feature_size):
#     return nn.Linear(p_size,feature_size)

In [41]:
# def decoder(self,feature,p,c):
#     p_size=p.shape[3]
#     feature_size=feature.shape[3]
#     fc_p2=fc_p(p_size,feature_size)
    
#     feature=self.feature_channels_change(feature)  
#     p = p.transpose(1, 2) #bs,3,2048
#     p=self.fc_p1(p) # bs,256,2048
#     p=fc_p2(p) #bs,256,7
#     p=p.unsqueeze(-1).repeat(1, 1, 1, feature_size)# bs,256,7,7
    
#     feature=torch.cat((feature, p), dim=1) #bs,512,7,7
    
#     pos_embed = self.positional_embeddings(feature)  
#     net=self.transformer(feature, self.query_embed.weight,pos_embed)[0] # out size =[2048, 512, 12]
    
#     net = self.before_last_layer(out).transpose(0,2)  #to create output [bs=12, 256, 2048]
#     #print("type",type(out),type(feature_c))
#     out = self.fc_out(self.actvn(self.bn(net, c)))
#     out = out.squeeze(1)
#     p_r = dist.Bernoulli(logits=out)
#     return out

In [42]:
# Intialize training
npoints = 1000
optimizer = optim.Adam(model.parameters(), lr=1e-4)
trainer = train.Train(                         ## set the trainig for training
    model, optimizer,
    device=device, input_type="img",
    vis_dir="out/img/onet/vis", threshold=0.2,
    eval_sample=False,
)


In [43]:
## all here to load check point and get info from the check point
checkpoint_io = CheckpointIO(out_dir, model=model, optimizer=optimizer)  ##save check poins
try:                                                            ##here to load checkpoint to cotinue to train i guess
    load_dict = checkpoint_io.load('model.pt')         ## so here model saved as .pt not .ckpt
except FileExistsError:
    load_dict = dict()
epoch_it = load_dict.get('epoch_it', -1)
it = load_dict.get('it', -1)
metric_val_best = load_dict.get(
    'loss_val_best', -np.inf)


print('Current best validation metric (%s): %.8f'
      % (model_selection_metric, metric_val_best))   ## model_selection_metric here is IoU metric


Current best validation metric (iou): -inf


In [44]:
###TODO: reintroduce or remove scheduler? - not delete here
# scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=4000,
#                                       gamma=0.1, last_epoch=epoch_it)


# Shorthands
print_every = 10
#checkpoint_every = cfg['training']['checkpoint_every']
checkpoint_every =50
#validate_every = cfg['training']['validate_every']
validate_every = 50
# visualize_every = cfg['training']['visualize_every']
visualize_every = 50
# Print model                                                   
nparameters = sum(p.numel() for p in model.parameters())
print(model)
print('Total number of parameters: %d' % nparameters)

OccupancyNetwork(
  (decoder): DecoderCBatchNorm(
    (fc_p): Conv1d(3, 256, kernel_size=(1,), stride=(1,))
    (block0): CResnetBlockConv1d(
      (bn_0): CBatchNorm1d(
        (conv_gamma): Conv1d(256, 256, kernel_size=(1,), stride=(1,))
        (conv_beta): Conv1d(256, 256, kernel_size=(1,), stride=(1,))
        (bn): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
      )
      (bn_1): CBatchNorm1d(
        (conv_gamma): Conv1d(256, 256, kernel_size=(1,), stride=(1,))
        (conv_beta): Conv1d(256, 256, kernel_size=(1,), stride=(1,))
        (bn): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
      )
      (fc_0): Conv1d(256, 256, kernel_size=(1,), stride=(1,))
      (fc_1): Conv1d(256, 256, kernel_size=(1,), stride=(1,))
      (actvn): ReLU()
    )
    (block1): CResnetBlockConv1d(
      (bn_0): CBatchNorm1d(
        (conv_gamma): Conv1d(256, 256, kernel_size=(1,), stride=(1,))
        (conv_beta): Conv1d(256,

In [45]:
torch.cuda.empty_cache()


In [46]:
epoch_it=2
while True:
    epoch_it += 1
    #scheduler.step()

    for batch in train_loader:
        it += 1
        loss = trainer.train_step(batch)
        logger.add_scalar('train/loss', loss, it)

        # Print output
        if print_every > 0 and (it % print_every) == 0:
            print('[Epoch %02d] it=%03d, loss=%.4f'
                  % (epoch_it, it, loss))

        # Visualize output
        if visualize_every > 0 and (it % visualize_every) == 0:
            print('Visualizing')
            trainer.visualize(data_vis)

        # Save checkpoint
        if (checkpoint_every > 0 and (it % checkpoint_every) == 0):
            print('Saving checkpoint')
            checkpoint_io.save('model.pt', epoch_it=epoch_it, it=it,
                               loss_val_best=metric_val_best)

        # Backup if necessary
        if (backup_every > 0 and (it % backup_every) == 0):
            print('Backup checkpoint')
            checkpoint_io.save('model_%d.pt' % it, epoch_it=epoch_it, it=it,
                               loss_val_best=metric_val_best)
        # Run validation
        if validate_every > 0 and (it % validate_every) == 0:
            eval_dict = trainer.evaluate(val_loader)
            metric_val = eval_dict[model_selection_metric]
            print('Validation metric (%s): %.4f'
                  % (model_selection_metric, metric_val))

            for k, v in eval_dict.items():
                logger.add_scalar('val/%s' % k, v, it)

            if (metric_val - metric_val_best) > 0:
                metric_val_best = metric_val
                print('New best model (loss %.4f)' % metric_val_best)
                checkpoint_io.save('model_best.pt', epoch_it=epoch_it, it=it,
                                   loss_val_best=metric_val_best)

[Epoch 03] it=000, loss=1817.8379
Visualizing
p_r: Bernoulli(logits: torch.Size([10, 32768]))
p_r.probs torch.Size([10, 32768])


100%|███████████████████████████████████████████████████████████████████████████████████| 10/10 [01:24<00:00,  8.47s/it]


Saving checkpoint
Backup checkpoint


100%|█████████████████████████████████████████████████████████████████████████████████| 365/365 [02:25<00:00,  2.51it/s]


Validation metric (iou): 0.0480
New best model (loss 0.0480)
[Epoch 03] it=010, loss=468.8732
[Epoch 03] it=020, loss=367.9352
[Epoch 03] it=030, loss=287.4140
[Epoch 03] it=040, loss=332.4796


KeyboardInterrupt: 

In [ ]:
torch.cuda.empty_cache()


In [ ]:
# model.p0_z

In [ ]:
tensorboard --logdir ./out/onet/logs --port 6006